In [1]:
import time
import os
import cv2

from lib.file.TIF import *
from lib.analysis.ROI_identification import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.file.TSV import RegionExporter
from lib.analysis.align import ImageAlign
from lib.file.DAT import TracesDAT
from lib.trace import Tracer
from lib.utilities import *
from lib.analysis.laminar_dist import *

In [17]:
# take every 'precursor' ROI file (listing the corners of a rectangular ROI) 
#    and generate a corresponding filled-in ROI file
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
enable_photoZ_interaction = False
# sets of files in each slice directory -- naming convention
precursor_file_token = '_roi_precursors' # e.g. slice3_roi_precursors.dat
possible_file_suffixes = ['_barrel', '_layer'] # e.g. slice3_roi_precursors_barrel.dat

# output files
rois_file_prefix = '_rois' # e.g. slice3_rois.dat

n_plots = 5
all_new_roi_files = []
for subdir, dirs, files in os.walk(data_dir):
    
    if 'notUsable' in subdir:
        continue
    if 'selected_zda' not in subdir:
        continue

    subdir += '/'

    for filename in files:
        fn = filename.split(".")[0]
        is_target_with_suffix = any(
            [fn.endswith(precursor_file_token + suf)
                for suf in possible_file_suffixes]
        )
        if fn.endswith(precursor_file_token) or is_target_with_suffix:
            
            la_file = subdir + filename

            lines = None
            if os.path.exists(la_file):
                print("\tProcessing", la_file)

                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                    
                num_roi_precursors = int((len(lines)-1) / 7)
                print("\tnum_roi_precursors:", num_roi_precursors)
                all_rois = []
                for i in range(num_roi_precursors):
                    start_line_no = 7*i+4
                
                    corners = [int(x) for x in lines[start_line_no:start_line_no+4]] # each 4 lines are diode numbers of corners
                    print("\t\tcorners:", corners)
                    layer_axes = LayerAxes(corners)
                    laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                    print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                          laminar_axis_2.get_unit_vector())

                    # make our own ROIs from the list of edges and write them to a .dat file
                    roi_cr = ROICreator(layer_axes, roi_width=27, stim_point_spacer=False)
                    rois = roi_cr.get_rois()  # list of LaminarROI objects
                    all_rois += rois

                # write these ROIs to file
                new_filename = filename.replace(precursor_file_token, rois_file_prefix)
                roi_cr.rois = all_rois
                roi_cr.n_rois_created = len(all_rois)
                written_filename = roi_cr.write_roi_file(subdir, new_filename)
                all_new_roi_files.append(written_filename)

print("\nFiles written:\n", len(all_new_roi_files))

	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-13-21\selected_zda/slice3_roi_precursors.dat
	num_roi_precursors: 5
		corners: [4003, 4745, 34, 59]
edge_pts [[34, 0], [59, 0]] axis_pts [[3, 50], [25, 59]]
laminar axis unit vectors: [0.5269396688616753, -0.8499026917123794] [0.49929862387917134, -0.8664299649667974]
		corners: [4746, 5239, 61, 78]
edge_pts [[61, 0], [78, 0]] axis_pts [[26, 59], [39, 65]]
laminar axis unit vectors: [0.5102019133149064, -0.8600546538736993] [0.5144957554275266, -0.8574929257125442]
		corners: [5321, 5731, 319, 1519]
edge_pts [[79, 3], [79, 18]] axis_pts [[41, 66], [51, 71]]
laminar axis unit vectors: [0.5164931702367068, -0.8562913085503299] [0.46712103309555225, -0.8841933840737239]
		corners: [3842, 3680, 2818, 1520]
edge_pts [[0, 46], [0, 19]] axis_pts [[2, 48], [18, 35]]
laminar axis unit vectors: [-0.7071067811865475, -0.7071067811865475] [-0.7474093186836597, -0.6643638388299198]
		corners: [2498, 1200, 18, 746]
edge_pts [[0, 1

	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-13-21\selected_zda\archived_rois/slice6_roi_precursors_barrel.dat
	num_roi_precursors: 1
		corners: [1080, 2940, 3920, 6330]
edge_pts [[0, 49], [10, 79]] axis_pts [[40, 13], [60, 36]]
laminar axis unit vectors: [-0.7432941462471664, 0.6689647316224497] [-0.758185196311683, 0.6520392688280474]
	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-13-21\selected_zda\archived_rois/slice7_roi_precursors_barrel.dat
	num_roi_precursors: 1
		corners: [4839, 2900, 2479, 66]
edge_pts [[79, 30], [66, 0]] axis_pts [[39, 60], [20, 36]]
laminar axis unit vectors: [0.8, -0.6] [0.7875050010075858, -0.6163082616581107]
	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/04-13-21\selected_zda\archived_rois/slice7_roi_precursors_layer.dat
	num_roi_precursors: 1
		corners: [4261, 3081, 6394, 6363]
edge_pts [[74, 79], [43, 79]] axis_pts [[21, 53], [41, 38]]
laminar axis unit vectors: [0.6270094052116179, 0